![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [1]:
import pandas as pd
import numpy as np

# Start coding here...

df_bank_marketing = pd.read_csv("bank_marketing.csv")
#print(df_bank_marketing.head())
#print(df_bank_marketing.columns)

# client dataframe
client_columns = client_columns = ['client_id', 'age', 'job', 'marital', 'education','credit_default', 'mortgage']

df_client = df_bank_marketing[client_columns].copy()
print(df_client.columns) # Verify

# campaign dataframe
campaign_columns = ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome','day','month']
df_campaign = df_bank_marketing[campaign_columns].copy()
print(df_campaign.columns) # verify

# economics dataframe

economic_columns = ['client_id','cons_price_idx','euribor_three_months']
df_economics = df_bank_marketing[economic_columns].copy()
print(df_economics.columns) # verify



Index(['client_id', 'age', 'job', 'marital', 'education', 'credit_default',
       'mortgage'],
      dtype='object')
Index(['client_id', 'number_contacts', 'contact_duration',
       'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome',
       'day', 'month'],
      dtype='object')
Index(['client_id', 'cons_price_idx', 'euribor_three_months'], dtype='object')


In [2]:
# Cleaning the data- Client dataframe 



#job	
# type -object	Client's type of job	action: Change "." to "_"

df_client['job'] = df_client['job'].astype(str).str.replace(".","_", regex= False)


#education	
# type -object	Client's level of education	action: Change "." to "_" and "unknown" to np.NaN

df_client['education'] = df_client['education'].astype(str).str.replace(".","_", regex= False)
df_client['education'] = df_client['education'].replace("unknown",np.NaN)

#credit_default	
# type -bool	Whether the client's credit is in default 
# action :Convert to boolean data type:1 if "yes", otherwise 0

df_client['credit_default'] =  df_client['credit_default'].astype(str).apply(lambda x: True if x.lower() == 'yes' else False)

#mortgage	
#type bool	"Whether the client has an existing mortgage (housing loan)"
# action: Convert to boolean data type:1 if "yes", otherwise 0

df_client['mortgage'] = df_client['mortgage'].astype(str).apply(lambda x : True if x.lower() == 'yes' else False)

# making sure datatypes are set according to specifications 

df_client = df_client.astype({
    'job' : 'object',
    'education' : 'object',
    'credit_default' : 'bool',
    'mortgage' :'bool'
})

# verify the changes - data types 
print(df_client[['job','education','credit_default','mortgage']].dtypes)

job               object
education         object
credit_default      bool
mortgage            bool
dtype: object


In [3]:
#  Cleaning campaign dataframe 


# previous_outcome	
# type bool	"Outcome of the previous campaign"	
# action: Convert to boolean data type:1 if "success", otherwise 0.

df_campaign['previous_outcome'] = df_campaign['previous_outcome'].astype(str).apply(lambda x: True if x.lower() == 'success' else False)

# campaign_outcome	
# type bool	"Outcome of the current campaign"
# action: Convert to boolean data type:1 if "yes", otherwise 0.

df_campaign['campaign_outcome'] = df_campaign['campaign_outcome'].astype(str).apply(lambda x: True if x.lower() == 'yes' else False)

# last_contact_date	
# type datetime	"Last date the client was contacted"
# action : Create from a combination of day, month, and a newly created year column (which should have a value of 2022); Format = "YYYY-MM-DD"


df_campaign['last_contact_date'] = pd.to_datetime(df_campaign['day'].astype(str) + '-' + df_campaign['month'].astype(str) + '-' + '-2022', format= "%d-%b-%Y", errors = 'coerce')
df_campaign = df_campaign.drop(columns=['day', 'month'])

df_campaign = df_campaign.astype({
    'client_id': 'int64',
    'number_contacts': 'int64',
    'contact_duration': 'int64',
    'previous_campaign_contacts': 'int64',
    'previous_outcome': 'bool',
    'campaign_outcome': 'bool'
    # 'last_contact_date' is already datetime[ns] from pd.to_datetime
})




In [4]:
# cleaning economic dataframe --> No changes specified
# verifying specified datatype 

print(df_economics.dtypes) # verified 

client_id                 int64
cons_price_idx          float64
euribor_three_months    float64
dtype: object


In [5]:
# saving the dataframe to csv files 

df_client.to_csv('client.csv', index = False)
df_campaign.to_csv('campaign.csv', index = False)
df_economics.to_csv('economics.csv', index = False)


In [6]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
